### Author(s): evankoh


### Description
This notebook demonstrates:
1. How to retrieve historical prices for a stock
2. Compute moving averages
3. Plot results with line charts

### Import all neccessary libraries/functions

In [1]:
import os
os.chdir('../')
from stockscafe.StocksCafeApi import StocksCafeApi
import stockscafe.indicators.momentum.MovingAverage as MA
import pandas as pd
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import init_notebook_mode, plot, iplot

### Parameters (If unfamiliar with python, then only change the values in this cell)

In [2]:
exchange = 'SGX'
symbol = 'D05'
name = 'DBS Bank'
shortMADays = 20
longMADays = 100
priceLookbackDays = 1000 # Max is 1000

### Declaration & Get Recent Prices

In [3]:
init_notebook_mode(connected=True)
sc = StocksCafeApi()
df = sc.getPrices(exchange, symbol, priceLookbackDays)
df.tail()

,change,change_percent,close,currency,date,high,low,open,volume
995,-0.130,-0.66%,19.59,SGD,11 Mar 2015,19.72,19.45,19.57,"3,694,400"
996,-0.010,-0.05%,19.72,SGD,10 Mar 2015,19.91,19.72,19.83,"4,632,000"
997,0.060,0.31%,19.73,SGD,9 Mar 2015,19.75,19.22,19.43,"4,738,000"
998,0.190,0.98%,19.67,SGD,6 Mar 2015,19.73,19.50,19.59,"4,052,600"
999,0.010,0.05%,19.48,SGD,5 Mar 2015,19.60,19.42,19.53,"2,889,200"


### Compute Moving Averages

In [4]:
df = MA.compute(df, shortMADays) # Compute and add to Data Frame
df = MA.compute(df, longMADays) # Compute and add to Data Frame
df.tail() # Check data - Try 'df.head()'

,change,change_percent,close,currency,date,high,low,open,volume,sma20,sma100
995,0.170,0.68%,25.18,SGD,25 Feb 2019,25.25,24.94,25.00,"3,049,000",24.5570,24.1546
996,-0.010,-0.04%,25.17,SGD,26 Feb 2019,25.26,24.98,25.26,"3,196,800",24.6045,24.1500
997,-0.020,-0.08%,25.15,SGD,27 Feb 2019,25.40,25.02,25.05,"3,894,500",24.6510,24.1420
998,-0.310,-1.23%,24.84,SGD,28 Feb 2019,25.25,24.84,25.25,"5,343,600",24.6820,24.1328
999,0.240,0.97%,25.08,SGD,1 Mar 2019,25.22,24.93,24.97,"2,153,700",24.7390,24.1259


### Plot Graph

In [5]:
# Good resource to learn about graph basics - https://www.kaggle.com/kanncaa1/plotly-tutorial-for-beginners

traceClose = go.Scatter(
                    x = df.date,
                    y = df.close,
                    mode = "lines",
                    name = "Close",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'))

traceShortMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{shortMADays}'],
                    mode = "lines",
                    name = f'SMA-{shortMADays}',
                    marker = dict(size = 10, color = 'rgba(0, 0, 255, 0.3)'))

traceLongMA = go.Scatter(
                    x = df.date,
                    y = df[f'sma{longMADays}'],
                    mode = "lines",
                    name = f'SMA-{longMADays}',
                    marker = dict(color = 'rgba(255, 0, 0, 0.3)'))

data = [traceClose, traceShortMA, traceLongMA]

layout = dict(title = f'{name} Moving Average',
              xaxis = dict(title = '', ticklen = 0, zeroline = False, 
                           showticklabels = True, range = [-10, len(df.index) + 10], dtick = 100, tickwidth = 1),
              autosize = True, showlegend = True
             )

fig = dict(data = data, layout = layout)

iplot(fig)

### Generate and store graph in html

In [ ]:
plot(fig, filename = f'output/{exchange}:{symbol}.html')